In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import argparse
from threading import Thread
import re
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np
import serial

from utils import depth_manager
from utils import projections

In [3]:
%matplotlib qt

In [4]:
# connect to the Serial
SERIAL_CONNECTION = serial.Serial('COM3', 9600)
time.sleep(2)

# Utils

In [5]:
def parse_serial(serial_msg):
    """
    Function to parse serial data to extract float values
    format 'x:19.34 y:23.01 z:-33.83' to x, y, z float values

    Args:
        - (str) string with format 'x:19.34 y:23.01 z:-33.83'
    Return:
        - (list) x, y, z float values
    """
    xyz_list = re.findall('[-+]?[0-9]*\.?[0-9]*', serial_msg)
    return [float(i) for i in filter(lambda item: item, xyz_list)]

In [6]:
def update_orientation(ser):
    """
    Function to integration the x data from the Gyroscope
    and update the global variable x_orientation with the new value

    Args:
        - (serial.Serial) serial to get the gyroscope data
    """
    global X_ORIENTATION

    while True:
        serial_msg_bytes = ser.readline()
        serial_msg = serial_msg_bytes.decode()
        dx, dy, dz = parse_serial(serial_msg)
        
        # The gyroscope values are in degrees-per-second
        # divide each value by the number of samples per second
        dx_normalized = dx / GYROSCOPE_SAMPLE_RATE

        # remove noise
        if abs(dx_normalized) > 0.004:
            # update orientation
            X_ORIENTATION = X_ORIENTATION - dx_normalized*1.25
            X_ORIENTATION = X_ORIENTATION%360

# Update orientation in real time

In [7]:
# initialize x orientation
X_ORIENTATION = 0

# 119 got from Arduino with IMU.gyroscopeSampleRate();
GYROSCOPE_SAMPLE_RATE = 119

In [8]:
# run the thread to update the x orientation in real time
Thread(target=update_orientation, args=(SERIAL_CONNECTION,)).start()

# Get depth image and do projection

In [9]:
INTERPRETER = depth_manager.get_tflite_interpreter(
    "https://tfhub.dev/intel/lite-model/midas/v2_1_small/1/lite/1?lite-format=tflite",
    "model/midas_v2_1_small.tflite")

# Input image

In [10]:
IMG_FROM_WEBCAM = True

if IMG_FROM_WEBCAM:
    VID = cv2.VideoCapture(0)
    # get first image for depth calculation
    ret, frame = VID.read()
    RGB_IMG = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
else:
    # Get input image from disk
    IMG = cv2.imread('images/dog.jpg')
    RGB_IMG = cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)

# Calcul projection in real time

In [11]:
PATH_TO_SAVE_SCENE = "3d_scene/"

In [48]:
POINTS_IN_3D = np.array([])
DEPTH_VALUES = []

PROJECT_DEPTH = True

DEGREES_INTERVAL = 300

ORIENTATIONS_DONE = []
ORIENTATIONS_TODO = [orientation for orientation
                     in range(0, 359, DEGREES_INTERVAL)]

DEPTH_MAP = depth_manager.run_tflite_interpreter(RGB_IMG, INTERPRETER)

OVERLAPS_IMG_DEPTH = {}
CORNERS_DISTANCE = {}
DEPTH_MAPS = {}
IMAGES = {}

PERCENTAGE_MARGIN_ON_DEPTH = 2

In [49]:
FIG_SIMULATION = plt.figure()

try:
    while True:
        if IMG_FROM_WEBCAM:
            ret, frame = VID.read()
            RGB_IMG = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        if ret:
            DEPTH_MAP, POINTS_IN_3D, DEPTH_VALUES, IMAGES, DEPTH_MAPS, OVERLAPS_IMG_DEPTH = \
                projections.plot_env(
                    FIG_SIMULATION, X_ORIENTATION, POINTS_IN_3D, DEPTH_VALUES,
                    RGB_IMG, INTERPRETER, ORIENTATIONS_DONE,
                    ORIENTATIONS_TODO, DEPTH_MAP, IMAGES, DEPTH_MAPS, OVERLAPS_IMG_DEPTH,
                    CORNERS_DISTANCE, per_mil_to_keep=1, project_depth=PROJECT_DEPTH,
                    percentage_margin_on_depth=PERCENTAGE_MARGIN_ON_DEPTH)

        # stop if all todo orientations were done
        if not ORIENTATIONS_TODO:
            projections.save_3d_scene(PATH_TO_SAVE_SCENE, POINTS_IN_3D, DEPTH_VALUES,
                                      IMAGES, DEPTH_MAPS, OVERLAPS_IMG_DEPTH)

            plt.gcf().clear()
            projections.plot_3d_scene(FIG_SIMULATION, POINTS_IN_3D, DEPTH_VALUES)
            break
except KeyboardInterrupt:
    pass

Min distance:  5
Max distance:  15
Min distance:  5
Max distance:  45


# Display saved scene

In [50]:
FIG_SIMULATION = plt.figure()
SAVED_POINTS_IN_3D, SAVED_DEPTH_VALUES, SAVED_IMAGES,\
    SAVED_DEPTH_MAPS, SAVED_OVERLAPS = projections.load_3d_scene("3d_scene/")

projections.plot_3d_scene(FIG_SIMULATION, SAVED_POINTS_IN_3D, SAVED_DEPTH_VALUES)

In [53]:
plt.imshow(IMAGES[0])